In [44]:
import tweepy
from datetime import datetime, timedelta

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/ballmdr/creds.json"

ACC_TOKEN = '2154665286-jZt7RxgxwPXPMwqDT1dNdVGAaxnJNSK2hiJ1DG8'
ACC_SECRET = 'nROZBHSWzSljFa7OKSt3RZFoWyoK4xmantmLQzBTFkWKB'
CONS_KEY = 'RJ7YxwWO9WjglaffUvfSVVwW2'
CONS_SECRET = 'EfcHwKxbTnYVkWbBXtPI1LCNz9P6CAyXaQKtaVrAgm4tK9hls0'

auth = tweepy.OAuthHandler(CONS_KEY, CONS_SECRET)
auth.set_access_token(ACC_TOKEN, ACC_SECRET)
api = tweepy.API(auth)
client = language.LanguageServiceClient()

In [47]:
today_datetime = datetime.today().now()
yesterday_datetime = today_datetime - timedelta(days=1)
today_date = today_datetime.strftime('%Y-%m-%d')
yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')

In [85]:
keywords = ['EURUSD', 'USDJPY', 'GBPUSD', 'USDCAD', 'USDCHF', 'AUDUSD', 'NZDUSD', 'EURAUD', 'EURCAD', 'EURCHF', 'EURGBP', 'EURJPY', 'EURNZD', 'GBPAUD', 'GBPCAD', 'GBPCHF', 'GBPJPY', 'GBPNZD', 'AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'NZDCAD', 'NZDCHF', 'NZDJPY', 'CADCHF', 'CAD/PY', 'CHFJPY']

for keyword in keywords:
    total_tweets = 100
    score = 0

    tweets = tweepy.Cursor(api.search, 
                                  q=keyword, 
                                  since=yesterday_date, 
                                  result_type='recent', 
                                  lang='en').items(total_tweets)
    for tweet in tweets:
        #print(tweet.text)
        #remove the username
        user_removed = re.sub(r'@[A-Za-z0-9]+','',tweet.text)
        #print('user_remove', user_removed)
        #remove link
        link_removed = re.sub('https?://[A-Za-z0-9./]+','',user_removed)
        #print('link_removed', link_removed)
        #delete number
        number_removed = re.sub('[^a-zA-Z]',' ',link_removed)
        #print('number_removed', number_removed)
        # convert all of the characters into lower space, then remove every unnecessary space.
        lower_case_tweet = number_removed.lower()
        tok = WordPunctTokenizer()
        words = tok.tokenize(lower_case_tweet)
        #print('words', words)
        clean_tweet = (' '.join(words)).strip()
        #print('clean_tweet', clean_tweet)

        document = types.Document(content=tweet.text, 
                                  type=enums.Document.Type.PLAIN_TEXT)
        #print('document', document)
        sentiment_score = client.analyze_sentiment(document=document).document_sentiment.score

        score += sentiment_score
        #print('Tweet: {}'.format(clean_tweet))
        #print('Score: {}\n'.format(sentiment_score))

    final_score = round((score / float(total_tweets)), 2)
    if final_score <= -0.25:
        print('%s %.2f NEGATIVE' % (keyword, final_score))
    elif final_score <= 0.25:
        print('%s %.2f NEUTRAL' % (keyword, final_score))
    else:
        print('%s %.2f POSITIVE' % (keyword, final_score))

EURUSD 0.02 NEUTRAL


InvalidArgument: 400 The language nl is not supported for document_sentiment analysis.